<a href="https://colab.research.google.com/github/codesongs/codestates_TP2/blob/main/RecVAE_recbole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/tp2'

Mounted at /content/drive


In [ ]:
# 데이터 가져오기
import os
import pandas as pd

def load_ratings(path):
    COL_NAME = ['userId','movieId','rating','timestamp']
    df = pd.read_csv(os.path.join(path,"ratings.dat"),sep='::', header=None, engine='python', names=COL_NAME)
    return df

def load_movies(path):
    COL_NAME = ['movieId','title','genres']
    df = pd.read_csv(os.path.join(path,"movies.dat"),sep='::', header=None, engine='python', names=COL_NAME, encoding = 'ISO-8859-1' )
    return df

def load_users(path):
    COL_NAME = ['userId','gender','age','Occupation','zip_code']
    df = pd.read_csv(os.path.join(path,"users.dat"),sep='::', header=None, engine='python', names=COL_NAME)
    return df

df_ratings = load_ratings(path)
df_movies = load_movies(path)
df_users = load_users(path)

In [ ]:
!pip install recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00


In [ ]:
!pip install bottleneck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/RUCAIBox/RecBole.git

Cloning into 'RecBole'...
remote: Enumerating objects: 22653, done.
remote: Counting objects: 100% (22653/22653), done.
remote: Compressing objects: 100% (7431/7431), done.
remote: Total 22653 (delta 15059), reused 22475 (delta 15033), pack-reused 0
Receiving objects: 100% (22653/22653), 18.75 MiB | 11.89 MiB/s, done.
Resolving deltas: 100% (15059/15059), done.


In [ ]:
!git clone https://github.com/ilya-shenbin/RecVAE.git

Cloning into 'RecVAE'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 47 (delta 9), reused 6 (delta 6), pack-reused 32
Receiving objects: 100% (47/47), 25.77 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
%cd /content/RecVAE

/content/RecVAE


In [ ]:
#!python preprocessing.py --dataset '/content/drive/MyDrive/tp2/ratings.dat' --output_dir '/content/' --threshold 3.5 --heldout_users 500

Traceback (most recent call last):
  File "/content/RecVAE/preprocessing.py", line 29, in <module>
    raw_data = pd.read_csv(dataset, header=0)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 605, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1442, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "/usr/local/lib/python3.10/dist-packages/pandas/io/parsers/readers.py", line 1735, in _make_engine
    self.handles = get_handle(
  F

In [ ]:
import os
import sys

import numpy as np
from scipy import sparse
import pandas as pd

dataset = '/content/drive/MyDrive/tp2/ratings.dat'
output_dir = '/content/'
threshold = 3.5
min_uc = 5
min_sc = 0
n_heldout_users = 1000

raw_data = pd.read_csv(dataset, sep='::', header=None, engine='python', names=['userId','movieId','rating','timestamp'])
raw_data = raw_data[raw_data['rating'] > threshold]
raw_data.head()


def get_count(tp, id):
    playcount_groupbyid = tp.groupby(id, as_index=True).size()
    return playcount_groupbyid


def filter_triplets(tp, min_uc=min_uc, min_sc=min_sc):
    if min_sc > 0:
        itemcount = get_count(tp, 'movieId')
        tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]

    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]

    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId')
    return tp, usercount, itemcount


raw_data, user_activity, item_popularity = filter_triplets(raw_data)

sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])

print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" %
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

unique_uid = user_activity.index

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

n_users = unique_uid.size

tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]

unique_sid = pd.unique(train_plays['movieId'])

show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(os.path.join(output_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

with open(os.path.join(output_dir, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)


def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)

    return data_tr, data_te


vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]

vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]

test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['userId']))
    sid = list(map(lambda x: show2id[x], tp['movieId']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])


train_data = numerize(train_plays)
train_data.to_csv(os.path.join(output_dir, 'train.csv'), index=False)

vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(output_dir, 'validation_tr.csv'), index=False)

vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(output_dir, 'validation_te.csv'), index=False)

test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(output_dir, 'test_tr.csv'), index=False)

test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(output_dir, 'test_te.csv'), index=False)

After filtering, there are 575272 watching events from 6034 users and 3533 movies (sparsity: 2.699%)
0 users sampled
0 users sampled


In [ ]:
# 시계열식 데이터 분할
import os
import sys

import numpy as np
from scipy import sparse
import pandas as pd

dataset = '/content/drive/MyDrive/tp2/ratings.dat'
output_dir = '/content/'
threshold = 3.5
min_uc = 5
min_sc = 0
n_heldout_users = 1000

raw_data = pd.read_csv(dataset, sep='::', header=None, engine='python', names=['userId','movieId','rating','timestamp'])
raw_data = raw_data[raw_data['rating'] > threshold]
raw_data.head()

# 데이터 정렬: 각 사용자별로 timestamp 기준으로 정렬
raw_data = raw_data.sort_values(by=['userId', 'timestamp'])

# Stratified Split 기반의 Time-based Split
def stratified_time_based_split(data, test_prop=0.2, val_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    train_list, val_tr_list, val_te_list, test_tr_list, test_te_list = [], [], [], [], []

    for _, group in data_grouped_by_user:
        n_items_u = len(group)

        test_size = int(n_items_u * test_prop)
        val_size = int(n_items_u * val_prop)
        train_size = n_items_u - val_size - test_size

        val_test_boundary = train_size + val_size

        train_list.append(group.iloc[:train_size])
        val_tr_list.append(group.iloc[train_size:val_test_boundary - test_size//2])
        val_te_list.append(group.iloc[val_test_boundary - test_size//2:val_test_boundary])
        test_tr_list.append(group.iloc[val_test_boundary:val_test_boundary + test_size//2])
        test_te_list.append(group.iloc[val_test_boundary + test_size//2:])

    return pd.concat(train_list), pd.concat(val_tr_list), pd.concat(val_te_list), pd.concat(test_tr_list), pd.concat(test_te_list)

train_data_raw, val_data_tr_raw, val_data_te_raw, test_data_tr_raw, test_data_te_raw = stratified_time_based_split(raw_data)

unique_uid = raw_data['userId'].unique()
unique_sid = raw_data['movieId'].unique()

# unique_sid.txt 파일 생성
with open(os.path.join(output_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

# unique_uid.txt 파일 생성
with open(os.path.join(output_dir, 'unique_uid.txt'), 'w') as f:
    for uid in unique_uid:
        f.write('%s\n' % uid)

profile2id = dict((uid, i) for (i, uid) in enumerate(unique_uid))
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))

# 데이터 numerize
def numerize(tp):
    uid = list(map(lambda x: profile2id[x], tp['userId']))
    sid = list(map(lambda x: show2id[x], tp['movieId']))
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

train_data = numerize(train_data_raw)
train_data.to_csv(os.path.join(output_dir, 'train.csv'), index=False)

val_data_tr = numerize(val_data_tr_raw)
val_data_tr.to_csv(os.path.join(output_dir, 'validation_tr.csv'), index=False)

val_data_te = numerize(val_data_te_raw)
val_data_te.to_csv(os.path.join(output_dir, 'validation_te.csv'), index=False)

test_data_tr = numerize(test_data_tr_raw)
test_data_tr.to_csv(os.path.join(output_dir, 'test_tr.csv'), index=False)

test_data_te = numerize(test_data_te_raw)
test_data_te.to_csv(os.path.join(output_dir, 'test_te.csv'), index=False)


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
from scipy import sparse
import pandas as pd
import os
import bottleneck as bn

def load_train_data(csv_file, n_items, n_users, global_indexing=False):
    tp = pd.read_csv(csv_file)

    n_users = n_users if global_indexing else tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data


def load_tr_te_data(csv_file_tr, csv_file_te, n_items, n_users, global_indexing=False):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    if global_indexing:
        start_idx = 0
        end_idx = len(unique_uid) - 1
    else:
        start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
        end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te


def get_data(dataset, global_indexing=False):
    unique_sid = list()
    with open(os.path.join(dataset, 'unique_sid.txt'), 'r') as f:
        for line in f:
            unique_sid.append(line.strip())

    unique_uid = list()
    with open(os.path.join(dataset, 'unique_uid.txt'), 'r') as f:
        for line in f:
            unique_uid.append(line.strip())

    n_items = len(unique_sid)
    n_users = len(unique_uid)

    train_data = load_train_data(os.path.join(dataset, 'train.csv'), n_items, n_users, global_indexing=global_indexing)


    vad_data_tr, vad_data_te = load_tr_te_data(os.path.join(dataset, 'validation_tr.csv'),
                                               os.path.join(dataset, 'validation_te.csv'),
                                               n_items, n_users,
                                               global_indexing=global_indexing)

    test_data_tr, test_data_te = load_tr_te_data(os.path.join(dataset, 'test_tr.csv'),
                                                 os.path.join(dataset, 'test_te.csv'),
                                                 n_items, n_users,
                                                 global_indexing=global_indexing)

    data = train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te
    data = (x.astype('float32') for x in data)

    return data


def ndcg(X_pred, heldout_batch, k=20):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG


def recall(X_pred, heldout_batch, k=20):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [ ]:
import numpy as np

import torch
from torch import optim

import random
from copy import deepcopy

from utils import get_data, ndcg, recall
from model import VAE

hidden_dim=512
latent_dim=256
batch_size=256
beta=None
gamma=0.005
lr=5e-4
n_epochs=50
n_enc_epochs=3
n_dec_epochs=1
not_alternating=False


seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda:0")
#device = torch.device("cpu")

data = get_data('/content/')
train_data, valid_in_data, valid_out_data, test_in_data, test_out_data = data


def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1

    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)

    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)

    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)


class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out

    def get_idx(self):
        return self._idx

    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)

    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]

    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)


def evaluate(model, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=256):
    metrics = deepcopy(metrics)
    model.eval()

    for m in metrics:
        m['score'] = []

    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):

        ratings_in = batch.get_ratings_to_dev()
        ratings_out = batch.get_ratings(is_out=True)

        ratings_pred = model(ratings_in, calculate_loss=False).cpu().detach().numpy()

        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf

        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()

    return [x['score'] for x in metrics]


def run(model, opts, train_data, batch_size, n_epochs, beta, gamma, dropout_rate):
    model.train()
    for epoch in range(n_epochs):
        for batch in generate(batch_size=batch_size, device=device, data_in=train_data, shuffle=True):
            ratings = batch.get_ratings_to_dev()

            for optimizer in opts:
                optimizer.zero_grad()

            _, loss = model(ratings, beta=beta, gamma=gamma, dropout_rate=dropout_rate)
            loss.backward()

            for optimizer in opts:
                optimizer.step()


model_kwargs = {
    'hidden_dim': hidden_dim,
    'latent_dim': latent_dim,
    'input_dim': train_data.shape[1]
}
# metrics = [{'metric': ndcg, 'k': 20}]
metrics = [{'metric': ndcg, 'k': 20}, {'metric': recall, 'k': 20}]

best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': batch_size,
    'beta': beta,
    'gamma': gamma
}

decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_params, lr=lr)
optimizer_decoder = optim.Adam(decoder_params, lr=lr)

for epoch in range(n_epochs):

    if not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_ndcg, train_recall = evaluate(model, train_data, train_data, metrics, 0.01)
    valid_ndcg, valid_recall = evaluate(model, valid_in_data, valid_out_data, metrics, 1)

    train_scores.append(train_ndcg)
    valid_scores.append(valid_ndcg)

    if valid_ndcg > best_ndcg:
        best_ndcg = valid_ndcg
        model_best.load_state_dict(deepcopy(model.state_dict()))

    print(f'epoch {epoch} | valid ndcg@20: {valid_ndcg:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@20: {train_ndcg:.4f} | ' +
          f'train recall@20: {train_recall:.4f} | valid recall@20: {valid_recall:.4f}')


test_metrics = [{'metric': ndcg, 'k': 20}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

epoch 0 | valid ndcg@20: 0.1266 | best valid: 0.1266 | train ndcg@20: 0.3263 | train recall@20: 0.2952 | valid recall@20: 0.1349
epoch 1 | valid ndcg@20: 0.1939 | best valid: 0.1939 | train ndcg@20: 0.4166 | train recall@20: 0.3722 | valid recall@20: 0.1990
epoch 2 | valid ndcg@20: 0.2327 | best valid: 0.2327 | train ndcg@20: 0.5055 | train recall@20: 0.4539 | valid recall@20: 0.2398
epoch 3 | valid ndcg@20: 0.2688 | best valid: 0.2688 | train ndcg@20: 0.5959 | train recall@20: 0.5419 | valid recall@20: 0.2800
epoch 4 | valid ndcg@20: 0.2858 | best valid: 0.2858 | train ndcg@20: 0.6381 | train recall@20: 0.5930 | valid recall@20: 0.3007
epoch 5 | valid ndcg@20: 0.2968 | best valid: 0.2968 | train ndcg@20: 0.6705 | train recall@20: 0.6223 | valid recall@20: 0.3146
epoch 6 | valid ndcg@20: 0.3072 | best valid: 0.3072 | train ndcg@20: 0.6839 | train recall@20: 0.6308 | valid recall@20: 0.3243
epoch 7 | valid ndcg@20: 0.3108 | best valid: 0.3108 | train ndcg@20: 0.7238 | train recall@20: 0

In [ ]:
## 시계열로 분리했을때 사용방법 ##
def recommend_top_n_movies_for_user(user_id, model, n=20):
    # Adjusting the user_id for zero-based indexing
    user_index = user_id - 1

    # 1. 해당 사용자의 데이터를 가져옵니다.
    user_data = test_in_data[user_index].toarray()
    user_data_tensor = torch.Tensor(user_data).to(device)

    # 2. 모델을 사용하여 사용자의 영화에 대한 평점을 예측합니다.
    with torch.no_grad():
        predicted_ratings = model(user_data_tensor, calculate_loss=False).cpu().numpy()

    # 3. 예측된 평점이 가장 높은 상위 n개의 영화의 인덱스를 얻습니다.
    recommended_movie_indices = np.argsort(predicted_ratings[0])[::-1][:n]

    return recommended_movie_indices

user_id = 60
recommended_movies = recommend_top_n_movies_for_user(user_id, model_best)
print(f"Recommended movies for user {user_id}: {recommended_movies}")


Recommended movies for user 60: [1025  233  123 1577  223   83 1669  229   66   69  362   86  110   77
  126 1034   40   52  523   20]


In [ ]:
# 영화의 총 개수를 movieId의 최대값으로 설정
total_items = int(df_movies['movieId'].max() + 1)

# 사용자 데이터 생성 함수
def get_user_data(ratings, userId, total_items=3468):  # total_items 기본 값을 모델의 첫 번째 레이어 크기로 설정
    user_ratings = ratings[ratings['userId'] == userId]
    user_data = np.zeros(total_items)
    for _, row in user_ratings.iterrows():
        if int(row['movieId']) < total_items:  # 이 조건을 추가하여 total_items 범위 내에서만 1로 설정
            user_data[int(row['movieId'])] = 1  # 시청한 영화를 1로 표시
    return user_data


# 주어진 사용자 데이터를 바탕으로 추천 리스트 생성
def get_recommendations_for_user(model, user_data_tensor, top_k=20):
    model.eval()
    with torch.no_grad():
        predictions = model(user_data_tensor)
    _, indices = torch.topk(predictions, top_k)
    return indices.cpu().numpy()

# userId를 입력받습니다.
userId = int(input("Enter the userId: "))

# 해당 userId의 시청 데이터를 추출
user_data = get_user_data(df_ratings, userId, total_items)
user_data_tensor = torch.Tensor(user_data).unsqueeze(0).to(device)

# 추천 리스트 생성
recommendation_list = get_recommendations_for_user(model_best, user_data_tensor)

print(f"Top 20 recommendations for user {userId}: {recommendation_list}")

Enter the userId: 1


RuntimeError: ignored

In [ ]:
def get_user_interaction_matrix(user_id, train_data, n_items):
    user_data = train_data[train_data['uid'] == user_id]
    interaction_matrix = np.zeros(n_items)

    for index, row in user_data.iterrows():
        interaction_matrix[row['sid']] = 1  # 혹은 해당 사용자의 평점이나 interaction 값을 설정할 수 있습니다.

    return interaction_matrix.reshape(1, -1)

# def recommend_top_n_movies_for_user(user_id, model, n_items, n=20):
#     # 1. 해당 사용자의 데이터를 가져옵니다.
#     user_data = get_user_interaction_matrix(user_id, train_data, n_items)
#     user_data_tensor = torch.Tensor(user_data).to(device)

#     # 2. 모델을 사용하여 사용자의 영화에 대한 평점을 예측합니다.
#     with torch.no_grad():
#         predicted_ratings = model(user_data_tensor, calculate_loss=False).cpu().numpy()

#     # 3. 예측된 평점이 가장 높은 상위 n개의 영화의 인덱스를 얻습니다.
#     recommended_movie_indices = np.argsort(predicted_ratings[0])[::-1][:n]

#     return recommended_movie_indices

# user_id = 1410  # 혹은 다른 사용자 ID
# n_items = len(unique_sid)  # 전체 영화의 수
# recommended_movies = recommend_top_n_movies_for_user(user_id, model_best, n_items)
# print(f"Recommended movies for user {user_id}: {recommended_movies}")

def recommend_top_n_movies_for_user(user_id, model, n_items, n=20):
    # 1. 해당 사용자의 데이터를 가져옵니다.
    user_data = get_user_interaction_matrix(user_id, train_data, n_items)
    user_data_tensor = torch.Tensor(user_data).to(device)

    # 2. 모델을 사용하여 사용자의 영화에 대한 평점을 예측합니다.
    with torch.no_grad():
        predicted_ratings = model(user_data_tensor, calculate_loss=False).cpu().numpy()

    # 3. 예측된 평점이 가장 높은 상위 n개의 영화의 인덱스를 얻습니다.
    recommended_movie_indices = np.argsort(predicted_ratings[0])[::-1][:n]

    return recommended_movie_indices, predicted_ratings

In [ ]:
# 9월 3일 수정
from scipy.sparse import vstack, hstack

# 각 사용자에 대한 valid 및 test 데이터를 합칩니다.
valid_combined_data = valid_in_data + valid_out_data
test_combined_data = test_in_data + test_out_data

# 모든 데이터를 합칩니다.
all_data = vstack([train_data, valid_combined_data, test_combined_data])

%cd /content/

with open("unique_uid.txt", "r") as file:
    unique_uid = file.readlines()
    unique_uid = [line.strip() for line in unique_uid]

with open("unique_sid.txt", "r") as file:
    unique_sid = file.readlines()
    unique_sid = [line.strip() for line in unique_sid]

# 주어진 userId를 all_data 인덱스로 변환합니다.
def get_index_from_userId(user_id, unique_uid):
    return np.where(unique_uid == user_id)[0][0]

# 주어진 실제 movieId를 all_data의 인덱스로 변환합니다.
def get_index_from_movieId(movie_id, unique_sid):
    return np.where(unique_sid == movie_id)[0][0]

# 사용자의 interaction matrix를 반환합니다.
def get_user_interaction_matrix(user_id, data, n_items, unique_uid):
    internal_uid = unique_uid.index(str(user_id))
    interaction_matrix = np.zeros(n_items)

    start_idx = data.indptr[internal_uid]
    end_idx = data.indptr[internal_uid + 1]
    item_indices = data.indices[start_idx:end_idx]

    interaction_matrix[item_indices] = 1

    return interaction_matrix.reshape(1, -1)


# # 사용자가 이미 시청한 영화의 인덱스 목록을 반환합니다.
# def get_watched_movie_indices(user_id, data, unique_uid):
#     # unique_uid에서 사용자 ID의 위치를 찾습니다.
#     internal_uid = unique_uid.index(str(user_id))

#     # 해당 위치의 사용자 정보를 all_data에서 가져옵니다.
#     start_idx = data.indptr[internal_uid]
#     end_idx = data.indptr[internal_uid + 1]
#     watched_indices = data.indices[start_idx:end_idx].tolist()

#     return watched_indices

# 사용자가 이미 시청한 영화의 인덱스 목록을 반환합니다.
def get_watched_movie_indices(user_id, data, unique_uid):
    # unique_uid에서 사용자 ID의 위치를 찾습니다.
    internal_uid = unique_uid.index(str(user_id))

    # 해당 위치의 사용자 정보를 df_ratings에서 가져옵니다.
    user_ratings = df_ratings[(df_ratings['userId'] == int(user_id))]
    watched_indices = [unique_sid.index(str(movie_id)) for movie_id in user_ratings['movieId'].values]

    return watched_indices


def recommend_top_n_movies_for_user(user_id, model, data, n_items, n=3883):
    # 1. 해당 사용자의 데이터를 가져옵니다.
    user_data = get_user_interaction_matrix(user_id, data, n_items, unique_uid)
    user_data_tensor = torch.Tensor(user_data).to(device)

    # 2. 모델을 사용하여 사용자의 영화에 대한 평점을 예측합니다.
    with torch.no_grad():
        predicted_ratings = model(user_data_tensor, calculate_loss=False).cpu().numpy()

    # 이미 시청한 영화를 제외합니다.
    watched_indices = get_watched_movie_indices(user_id, data, unique_uid)

    # 3. 예측된 평점이 가장 높은 상위 n개의 영화의 인덱스를 얻습니다.
    top_n_indices = np.argsort(predicted_ratings[0])[::-1]
    top_n_recommended = [idx for idx in top_n_indices if idx not in watched_indices][:n]

    return top_n_recommended, predicted_ratings


/content


In [ ]:
user_id = 56
n_items = len(unique_sid)
recommended_movies, predicted_ratings = recommend_top_n_movies_for_user(user_id, model_best, all_data, n_items)

# show2id 딕셔너리의 반전 버전을 만듭니다.
id2show = {v: k for k, v in show2id.items()}

# 추천된 영화의 실제 movieId 및 예측된 평점을 가져옵니다.
movie_ids = [id2show[idx] for idx in recommended_movies]
predicted_scores = [predicted_ratings[0][idx] for idx in recommended_movies]

# df_movies에서 해당 movieId에 대한 title과 장르를 가져옵니다.
titles = [df_movies[df_movies['movieId'] == movie_id]['title'].iloc[0] for movie_id in movie_ids]
genres = [df_movies[df_movies['movieId'] == movie_id]['genres'].iloc[0] for movie_id in movie_ids]

# 결과를 DataFrame으로 출력합니다.
result_df = pd.DataFrame({
    'movieId': movie_ids,
    'title': titles,
    'genres': genres,
    'predicted_score': predicted_scores
})

result_df


,movieId,title,genres,predicted_score
0,1265,Groundhog Day (1993),Comedy|Romance,5.433919
1,356,Forrest Gump (1994),Comedy|Romance|War,5.080958
2,2599,Election (1999),Comedy,4.950103
3,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller,4.831883
4,223,Clerks (1994),Comedy,4.782913
...,...,...,...,...
3396,3924,Pajama Party (1964),Comedy,-7.172233
3397,3875,"Devil Rides Out, The (1968)",Horror,-7.325182
3398,3642,In Old California (1942),Western,-7.391348
3399,3488,"Hideous Sun Demon, The (1959)",Horror|Sci-Fi,-7.700673


In [ ]:
result_df['movieId'].to_csv('movieId.csv', index=False)
from google.colab import files
files.download('movieId.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(all_data)

In [ ]:
len(df_ratings[df_ratings['userId']==56].movieId)

67

In [ ]:
len(df_ratings[(df_ratings['userId'] == int(56)) & (df_ratings['rating'] > 3)])

50

In [ ]:
# 사용자 56의 모든 movieId 가져오기
user_1_movieIds = df_ratings[df_ratings['userId'] == 56]['movieId'].tolist()

# result_df에서 해당 movieId와 일치하는 행을 필터링하여 출력
matching_rows = result_df[result_df['movieId'].isin(user_1_movieIds)]
matching_rows

,movieId,title,genres,predicted_score


In [ ]:
df_users[(df_users.gender=='M') & (df_users.age==35)]

,userId,gender,age,Occupation,zip_code
6,7,M,35,1,06810
13,14,M,35,0,60126
22,23,M,35,0,90049
28,29,M,35,7,33407
55,56,M,35,20,60440
...,...,...,...,...,...
6009,6010,M,35,0,79606
6010,6011,M,35,15,80538
6011,6012,M,35,15,02871
6017,6018,M,35,1,48906


In [ ]:
model_best

VAE(
  (encoder): Encoder(
    (fc1): Linear(in_features=3468, out_features=512, bias=True)
    (ln1): LayerNorm((512,), eps=0.1, elementwise_affine=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (ln2): LayerNorm((512,), eps=0.1, elementwise_affine=True)
    (fc3): Linear(in_features=512, out_features=512, bias=True)
    (ln3): LayerNorm((512,), eps=0.1, elementwise_affine=True)
    (fc4): Linear(in_features=512, out_features=512, bias=True)
    (ln4): LayerNorm((512,), eps=0.1, elementwise_affine=True)
    (fc5): Linear(in_features=512, out_features=512, bias=True)
    (ln5): LayerNorm((512,), eps=0.1, elementwise_affine=True)
    (fc_mu): Linear(in_features=512, out_features=256, bias=True)
    (fc_logvar): Linear(in_features=512, out_features=256, bias=True)
  )
  (prior): CompositePrior(
    (encoder_old): Encoder(
      (fc1): Linear(in_features=3468, out_features=512, bias=True)
      (ln1): LayerNorm((512,), eps=0.1, elementwise_affine=True)
      (fc2

In [ ]:
!python run.py --dataset '/content/'

epoch 0 | valid ndcg@100: 0.1359 | best valid: 0.1359 | train ndcg@100: 0.2629
epoch 1 | valid ndcg@100: 0.2053 | best valid: 0.2053 | train ndcg@100: 0.3691
epoch 2 | valid ndcg@100: 0.2592 | best valid: 0.2592 | train ndcg@100: 0.4483
epoch 3 | valid ndcg@100: 0.2973 | best valid: 0.2973 | train ndcg@100: 0.5068
epoch 4 | valid ndcg@100: 0.3219 | best valid: 0.3219 | train ndcg@100: 0.5660
epoch 5 | valid ndcg@100: 0.3390 | best valid: 0.3390 | train ndcg@100: 0.6028
epoch 6 | valid ndcg@100: 0.3566 | best valid: 0.3566 | train ndcg@100: 0.6277
epoch 7 | valid ndcg@100: 0.3672 | best valid: 0.3672 | train ndcg@100: 0.6562
epoch 8 | valid ndcg@100: 0.3735 | best valid: 0.3735 | train ndcg@100: 0.6724
epoch 9 | valid ndcg@100: 0.3798 | best valid: 0.3798 | train ndcg@100: 0.6865
epoch 10 | valid ndcg@100: 0.3831 | best valid: 0.3831 | train ndcg@100: 0.7004
epoch 11 | valid ndcg@100: 0.3876 | best valid: 0.3876 | train ndcg@100: 0.7100
epoch 12 | valid ndcg@100: 0.3932 | best valid: 0.

In [ ]:
from recbole.quick_start import run_recbole

parameter_dict = {
   'train_neg_sample_args': None,
}
run_recbole(model='RecVAE', dataset='ml-100k', config_dict=parameter_dict)

Evaluate   : 100%|███████████████████████████████████████████████| 472/472 [00:01<00:00, 259.45it/s]


{'best_valid_score': 0.4207,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.2345),
              ('mrr@10', 0.4207),
              ('ndcg@10', 0.2544),
              ('hit@10', 0.7847),
              ('precision@10', 0.1704)]),
 'test_result': OrderedDict([('recall@10', 0.2656),
              ('mrr@10', 0.5152),
              ('ndcg@10', 0.3143),
              ('hit@10', 0.7996),
              ('precision@10', 0.2087)])}

In [ ]:
vad_data_te

,uid,sid
0,4250,246
1,4250,69
2,4250,31
3,4250,88
4,4250,285
...,...,...
18021,4656,88
18022,4656,33
18023,4656,44
18024,4656,77
